In [ ]:
import numpy as np
import pandas as pd

In [26]:
callender_data = pd.read_csv('../data/raw-data/economic_calendar_19_24.csv', index_col='Date')
callender_data.head()

,Day,Hour,Country,Event,Actual,Previous,Consensus,Forecast
Date,,,,,,,,
01/01/19,Tuesday,20:45,CN,Caixin Manufacturing PMI DEC,49.7,50.2,50.1,50.1
01/04/19,Friday,8:30,US,Non Farm Payrolls DEC,312K,176K,177K,165K
01/06/19,Sunday,NaN,CN,US-China Trade Talks,NaN,NaN,NaN,NaN
01/07/19,Monday,10:00,CA,Ivey PMI s.a DEC,59.7,57.2,56.8,56.7
01/07/19,Monday,NaN,CN,US-China Trade Talks,NaN,NaN,NaN,NaN


We will focus on daily or 4-hourly charts at most, meaning the exact time of the news is not relevant for now, so it will be removed.

Additionally, the columns `Actual`, `Previous`, `Consensus`, and `Forecast` will be dropped, as we will allow the model to infer their impact on its own(merged data).

In [27]:
callender_data.drop(['Actual'], axis=1, inplace=True)
callender_data.drop('Previous', axis=1, inplace=True)
callender_data.drop('Forecast', axis=1, inplace=True)
callender_data.drop('Consensus', axis=1, inplace=True)
callender_data.drop('Hour', axis=1, inplace=True)

callender_data.head()

,Day,Country,Event
Date,,,
01/01/19,Tuesday,CN,Caixin Manufacturing PMI DEC
01/04/19,Friday,US,Non Farm Payrolls DEC
01/06/19,Sunday,CN,US-China Trade Talks
01/07/19,Monday,CA,Ivey PMI s.a DEC
01/07/19,Monday,CN,US-China Trade Talks


Next, we will apply one-hot encoding to this dataframe.

In [37]:
callender_oneHotEncoded= pd.get_dummies(callender_data, columns=['Day','Country', 'Event'], prefix=['Day', 'Country', 'Event'])
callender_oneHotEncoded.sort_index(inplace=True)
callender_oneHotEncoded.head()

,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Country_CA,Country_CN,Country_EA,...,Event_Unemployment Rate DEC,Event_Unemployment Rate FEB,Event_Unemployment Rate JAN,Event_Unemployment Rate JUL,Event_Unemployment Rate JUN,Event_Unemployment Rate MAR,Event_Unemployment Rate NOV,Event_Unemployment Rate OCT,Event_Unemployment Rate SEP,Event_Winter Economy Plan
Date,,,,,,,,,,,,,,,,,,,,,
01/01/19,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
01/01/20,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
01/02/23,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
01/03/20,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
01/03/21,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


We notice that some dates are duplicated (having more than one event) and some are missing (no events for that day).

For the second issue, we do not need to address it now, as the missing dates will naturally appear as a result of the final merge.

In [36]:
callender_merged = callender_oneHotEncoded.groupby(callender_oneHotEncoded.index).any()
callender_merged

,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Country_CA,Country_CN,Country_EA,...,Event_Unemployment Rate DEC,Event_Unemployment Rate FEB,Event_Unemployment Rate JAN,Event_Unemployment Rate JUL,Event_Unemployment Rate JUN,Event_Unemployment Rate MAR,Event_Unemployment Rate NOV,Event_Unemployment Rate OCT,Event_Unemployment Rate SEP,Event_Winter Economy Plan
Date,,,,,,,,,,,,,,,,,,,,,
01/01/19,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
01/01/20,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
01/02/23,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
01/03/20,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
01/03/21,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/23/22,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12/30/19,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
12/30/20,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
callender_merged = callender_merged.astype(int)
callender_merged.head()
callender_merged.to_csv('../data/processed-data/economic_calendar_19_24.csv')

In [43]:
callender_merged.columns.to_numpy()

array(['Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday',
       'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday', 'Country_CA',
       'Country_CN', 'Country_EA', 'Country_GB', 'Country_JP',
       'Country_RU', 'Country_US',
       'Event_20th National Congress of the Chinese Communist Party',
       'Event_Autumn Budget', 'Event_Autumn Statement',
       'Event_Balance of Trade\xa005', 'Event_Balance of Trade\xa0APR',
       'Event_Balance of Trade\xa0AUG', 'Event_Balance of Trade\xa0DEC',
       'Event_Balance of Trade\xa0FEB', 'Event_Balance of Trade\xa0JAN',
       'Event_Balance of Trade\xa0JAN-FEB',
       'Event_Balance of Trade\xa0JUL', 'Event_Balance of Trade\xa0JUN',
       'Event_Balance of Trade\xa0MAR', 'Event_Balance of Trade\xa0NOV',
       'Event_Balance of Trade\xa0OCT', 'Event_Balance of Trade\xa0SEP',
       'Event_BoC Interest Rate Decision',
       'Event_BoC Monetary Policy Report',
       'Event_BoE Gov Bailey Speech\xa0',
       'Event_BoE Interest Rate D

Note: This is not the final dataset for model training. We will **reduce** it further. 

Possible ways to reduce the number of columns:
- Group similar events into broader categories.
- Remove events that occur very infrequently.
- Analyze the impact of frequent events and merge those with similar effects.
- Drop columns with low variance (e.g., columns with mostly zeros).
- Use dimensionality reduction techniques like PCA to combine correlated columns.
- we might get rid of the weekend during the merging since we can trade on weekends